In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import pandas as pd
from os import walk
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import scipy.io as sio
import pickle
import h5py

In [ ]:
include_wave = 'no'
modelID = 3
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
duration = '24M'

In [ ]:
def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (52803 * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (52803 * 1.0)

inputs = tf.keras.layers.Input(shape=(13,))

hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0002), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
# model.summary()

In [ ]:
file = h5py.File('Weights/02_PredictionModel/Model%d_IncWave%s_%s.h5' % (modelID, include_wave, duration), 'r')
weight = []
for i in range(len(file.keys())):
   weight.append(file['weight' + str(i)][:])
model.set_weights(weight)

In [ ]:
def compute_predictions_pbnn(model, samples):
    prediction_distribution= model(samples)
    prediction_mean = np.squeeze(prediction_distribution.mean().numpy())/10
    prediction_stdv = np.squeeze(prediction_distribution.stddev().numpy())/10

    # The 95% CI is computed as mean ± (1.96 * stdv)
    upper = (prediction_mean + (1.96 * prediction_stdv))
    lower = (prediction_mean - (1.96 * prediction_stdv))

    
    return prediction_mean, prediction_stdv, upper, lower

def loglikelihood(y, loc, scale):
    dist = tfp.distributions.Normal(loc, scale)
    return dist.log_prob(y)

In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
fig1, ax1 = plt.subplots(1, figsize=(8.5*cm, 7.5*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.2, right=.98, top=0.98, bottom=0.3, hspace = 0.65, wspace=0.15)
fig2, ax2 = plt.subplots(1, figsize=(8.5*cm, 7.5*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.2, right=.98, top=0.98, bottom=0.3, hspace = 0.65, wspace=0.15)

input_files = ['train_input','test_input','mp01df_input','mp02df_input']
output_files = ['train_output','test_output','mp01df_output','mp02df_output']
# input_files = ['test_input']
# output_files = ['test_output']
for ind in range(len(input_files)):
    test_input = pd.read_pickle('DATA/%s'%(input_files[ind]))
    test_output = pd.read_pickle('DATA/%s'%(output_files[ind]))
    index = test_input.columns

    # Normlaization of input data
    # Data normalization according to training dataset/ model
    filehandler = open('Weights/Norm', 'rb') 
    std_scaler = pickle.load(filehandler)
    inputn = pd.DataFrame(std_scaler.transform(test_input), columns=test_input.columns) 
    outputn = test_output/10**6  # 
    
    # Retrive features based on the modelID
    index1 = pd.core.indexes.base.Index([]) # create a blank index array
    if modelID == 3: # Acc77
        index1 = index1.append(index[[7,8,13,14,19,20]])
    index1 = index1.append(index[21:]) # SCADA
    X = inputn[index1].values
    Y = outputn.values
    
    nsim = 10000
    DEM_tl = np.zeros([len(X)])
    DEM_tn = np.zeros([len(X)])
    DEM_tl2 = np.zeros([len(X)])
    DEM_tn2 = np.zeros([len(X)])
    mu_tl = np.zeros([len(X)])
    mu_tn = np.zeros([len(X)])
    mu_tl2 = np.zeros([len(X)])
    mu_tn2 = np.zeros([len(X)])
    sigma_tl = np.zeros([len(X)])
    sigma_tn = np.zeros([len(X)])
    sigma_tl2 = np.zeros([len(X)])
    sigma_tn2 = np.zeros([len(X)])
    ll_tl = np.zeros([len(X)])
    ll_tn = np.zeros([len(X)])
    for j in range(nsim):
#         np.random.seed(j)
        dems = model.predict(X)
        DEM_tl += dems[:,0]/10
        DEM_tn += dems[:,1]/10
        DEM_tl2 += (dems[:,0]/10)**2
        DEM_tn2 += (dems[:,1]/10)**2
        prediction_mean, prediction_stdv, upper, lower = compute_predictions_pbnn(model, X)
        mu_tl += prediction_mean[:,0]
        mu_tn += prediction_mean[:,1]
        mu_tl2 += (prediction_mean[:,0])**2
        mu_tn2 += (prediction_mean[:,1])**2
        sigma_tl += (prediction_stdv[:,0])**2
        sigma_tn += (prediction_stdv[:,1])**2
        sigma_tl2 += ((prediction_stdv[:,0])**2)**2
        sigma_tn2 += ((prediction_stdv[:,1])**2)**2
        lls = loglikelihood(Y, prediction_mean, prediction_stdv)
        ll_tl += lls[:,0]
        ll_tn += lls[:,1]
    
    
    M_tl = (DEM_tl/nsim)
    M_tn = (DEM_tn/nsim)
    V_tl = (DEM_tl2/nsim)-(DEM_tl/nsim)**2
    V_tn = (DEM_tn2/nsim)-(DEM_tn/nsim)**2
    
    Mmu_tl = (mu_tl/nsim)
    Mmu_tn = (mu_tn/nsim)
    Vmu_tl = (mu_tl2/nsim)-(mu_tl/nsim)**2
    Vmu_tn = (mu_tn2/nsim)-(mu_tn/nsim)**2
    
    Msigma_tl = sigma_tl/nsim
    Msigma_tn = sigma_tn/nsim
    Vsigma_tl = (sigma_tl2/nsim)-(sigma_tl/nsim)**2 
    Vsigma_tn = (sigma_tn2/nsim)-(sigma_tn/nsim)**2 
    
    LL_tl = ll_tl.numpy()/nsim
    LL_tn = ll_tn.numpy()/nsim
    sio.savemat('%s_7metrics_10000.mat'%(input_files[ind]),{'M_tl':M_tl, 'M_tn':M_tn, 
                                                      'V_tl':V_tl, 'V_tn':V_tn,
                                                      'Mmu_tl':Mmu_tl, 'Mmu_tn':Mmu_tn, 
                                                      'Vmu_tl':Vmu_tl, 'Vmu_tn':Vmu_tn, 
                                                      'Msigma_tl':Msigma_tl, 'Msigma_tn':Msigma_tn,
                                                      'Vsigma_tl':Vsigma_tl, 'Vsigma_tn':Vsigma_tn,
                                                      'LL_tl':LL_tl, 'LL_tn':LL_tn})

#     print(LL_tl)
    print('MU',Vmu_tl/V_tl)
    print('AU',sigma_tl/nsim/V_tl)
    print('MU Fraction',Vmu_tl/V_tl)
    print('AU Fraction',sigma_tl/nsim)
    bp1 = ax1.boxplot(np.transpose(V_tl), positions = [ind], patch_artist = True, widths = 0.35, meanline = True, 
                whis = [2.5,97.5], boxprops=dict(facecolor='#deebf7'), showfliers=False, showmeans=True, 
                medianprops = dict(linewidth=0, ls='-'), meanprops=dict(color='red'))
    bp2 = ax1.boxplot(np.transpose(V_tn), positions = [ind+0.35], patch_artist = True, widths = 0.35, meanline = True, 
                whis = [2.5,97.5], boxprops=dict(facecolor='#3182bd'), showfliers=False, showmeans=True,
                medianprops = dict(linewidth=0, ls='-'), meanprops=dict(color='red'))

    bp3 = ax2.boxplot(np.transpose(Vmu_tl), positions = [ind], patch_artist = True, widths = 0.35, meanline = True, 
                whis = [2.5,97.5], boxprops=dict(facecolor='#deebf7'), showfliers=False, showmeans=True, 
                medianprops = dict(linewidth=0, ls='-'), meanprops=dict(color='red'))
    bp4 = ax2.boxplot(np.transpose(Vmu_tn), positions = [ind+0.35], patch_artist = True, widths = 0.35, meanline = True, 
                whis = [2.5,97.5], boxprops=dict(facecolor='#3182bd'), showfliers=False, showmeans=True, 
                medianprops = dict(linewidth=0,  ls='-'), meanprops=dict(color='red'))

ax1.legend([bp1["boxes"][0], bp2["boxes"][0]], [r'DEM$_{tl}$', r'DEM$_{tn}$'], loc='upper left')
# ax1.set_ylim([-1, 1])
ax1.set_xticks([0.2,1.2,2.2,3.2])
ax1.set_xticklabels(['Fleet-leader (train)','Fleet-leader (test)','MP01 (test)','MP02 (test)'], horizontalalignment= 'right', rotation=45)     
ax1.set_ylabel(r'$\mathrm{\mathbf{\mathbb{H}}}}$ [DEM]')


ax2.legend([bp3["boxes"][0], bp4["boxes"][0]], [r'DEM$_{tl}$', r'DEM$_{tn}$'], loc='lower left')
ax2.set_xticks([0.2,1.2,2.2,3.2])
ax2.set_xticklabels(['Fleet-leader (train)','Fleet-leader (test)','MP01 (test)','MP02 (test)'], horizontalalignment= 'right', rotation=45)   
ax2.set_ylabel(r'$\mathrm{\mathbf{\mathbb{E}}}}$ [$\mathrm{\mathbf{\mathcal{L}}}}$]')
